#API 호출 & 파싱

openapi 호출 후 파싱하는 과정

api 호출이 종종 원활하지 않아 한번 저장해서 사용하였음

In [ ]:
import langchain
from urllib import request
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
from xml.etree import ElementTree #gpt가 이해할 수 있도록 XML -> json 파싱을 위함
import pandas as pd
import json
import os

# API 엔드포인트와 서비스 키
url = "http://api.kcisa.kr/openapi/API_TOU_050/request"
skey = "e5bd6f7d-e3ee-4a21-80ae-af05ad82af82"
params = '?' +'serviceKey='+ skey

request = Request(url + params)
request.get_method = lambda: 'GET'  # GET: 서버로부터 정보를 조회하기 위해 사용.
#print(response_body)  #너무 많은 데이터, 출력 중단됨.
print(url+params)


root = ElementTree.fromstring(response_body)

data_list=[]
for item in root.iter('item'):  #xml->딕셔너리
  item_dict = {}
  item_dict['title'] = item.find('title').text
  item_dict['issuedDate'] = item.find('issuedDate').text
  item_dict['category1'] = item.find('category1').text
  item_dict['category2'] = item.find('category2').text
  item_dict['category3'] = item.find('category3').text
  item_dict['description'] = item.find('description').text
  item_dict['tel'] = item.find('tel').text
  item_dict['url'] = item.find('url').text
  item_dict['address'] = item.find('address').text
  item_dict['coordinates'] = item.find('coordinates').text
  item_dict['charge'] = item.find('charge').text
  data_list.append(item_dict)

# description에 '동반불가'가 포함된 을 삭제
data_list = [item for item in data_list if '동반불가' not in item['description']]

df = pd.DataFrame(data_list)  # 딕셔너리 -> df

df.to_csv(os.path.join("/content/drive/MyDrive/캡스톤디자인1/반려동물chat", 'api_data.csv'), index=False)  #df 저장

#json_output = json.dumps(data_list, indent=1) # 딕셔너리 -> json

In [ ]:
print(url+params)

http://api.kcisa.kr/openapi/API_TOU_050/request?serviceKey=e5bd6f7d-e3ee-4a21-80ae-af05ad82af82


# 전처리

* 반려동물동반불가시설 제거
* 전부 결측값인 category3 제거
* 나머지 결측치 '정보없음'으로 대체

In [ ]:
data_list = [item for item in data_list if '동반불가' not in item['description']]
df = pd.DataFrame(data_list)  # 딕셔너리 -> df
df = df.drop('category3', axis=1)
df = df.fillna('정보없음')  #None -> '정보없음'

df.to_csv(os.path.join("/content/drive/MyDrive/캡스톤디자인1/반려동물chat", 'api_data.csv'), index=False) #저장

In [ ]:
df

,title,issuedDate,category1,category2,description,tel,url,address,coordinates,charge
0,헬로독하이캣,2022-11-30,반려동물 서비스,반려동물용품,운영시간 : 월~토 10:30~21:00 | 휴무일 : 매주 일요일 | 주차가능 |...,0507-1309-9590,정보없음,(08839) 서울특별시 관악구 신림로 108,"N37.4711305, E126.937598",입장(이용료)가격 정보 : 변동 | 애견 동반 추가 요금 : 없음
1,공존하개,2022-11-30,반려동물 서비스,반려동물용품,운영시간 : 매일 00:00~24:00 | 휴무일 : 연중무휴 | 주차가능 | 반려...,0507-1348-3455,http://www.instagram.com/coexistence_dog_cat,(16836) 경기도 용인시 수지구 정평로 120,"N37.32417367, E127.093943",입장(이용료)가격 정보 : 없음 | 애견 동반 추가 요금 : 없음
2,관문체육공원,2022-11-30,반려동반여행,여행지,운영시간 : 매일 00:00~24:00 | 휴무일 : 연중무휴 | 주차가능 | 반려...,02-504-7300,정보없음,(13830) 경기도 과천시 중앙로 294,"N37.44136421, E126.996346",입장(이용료)가격 정보 : 없음 | 애견 동반 추가 요금 : 없음
3,관악그랜드약국,2022-11-30,반려의료,동물약국,"운영시간 : 월~금 09:00~20:00, 토 09:00~17:00 | 휴무일 : ...",02-877-1023,정보없음,(08846) 서울특별시 관악구 신림로 141,"N37.47159171, E126.933442",입장(이용료)가격 정보 : 변동 | 애견 동반 추가 요금 : 없음
4,공룡알화석산지,2022-11-30,반려동반여행,여행지,운영시간 : 화~일 09:00~17:00 | 휴무일 : 매주 월요일 | 주차가능 |...,031-357-3951,http://dinopia.hscity.go.kr/user/index.jsp,(18235) 경기도 화성시 송산면 공룡로 659,"N37.26222503, E126.747071",입장(이용료)가격 정보 : 없음 | 애견 동반 추가 요금 : 없음
...,...,...,...,...,...,...,...,...,...,...
8486,꽃길약국,2022-11-30,반려의료,동물약국,"운영시간 : 월~금 09:30~20:00, 토 09:30~18:00 | 휴무일 : ...",032-428-0858,정보없음,(21674) 인천광역시 남동구 소래역남로 20,"N37.39809447, E126.734742",입장(이용료)가격 정보 : 변동 | 애견 동반 추가 요금 : 없음
8487,꽃냥,2022-11-30,반려동물 서비스,미용,운영시간 : 화~일 10:00~20:00 | 휴무일 : 매주 월요일 | 주차가능 |...,0507-1356-4051,https://blog.naver.com/f-cat,(14037) 경기도 안양시 만안구 태평로60번길 42,"N37.39633325, E126.930747",입장(이용료)가격 정보 : 변동 | 애견 동반 추가 요금 : 없음
8488,꽃다비펫,2022-11-30,반려동물 서비스,미용,운영시간 : 매일 10:00~19:00 | 휴무일 : 연중무휴 | 주차가능 | 반려...,032-468-4656,http://blog.naver.com/ksbabc_,(22240) 인천광역시 미추홀구 주승로197번길 13-35,"N37.44573791, E126.691666",입장(이용료)가격 정보 : 소형견 전체미용 35000원 | 애견 동반 추가 요금 : 없음
8489,김포하나동물병원,2022-11-30,반려동물 서비스,반려동물용품,"운영시간 : 월~금 10:00~20:00, 토 10:30~19:00, 일 10:00...",031-997-8275,https://blog.naver.com/ohmoo,(10108) 경기도 김포시 관순로 4,"N37.62147372, E126.715861",입장(이용료)가격 정보 : 변동 | 애견 동반 추가 요금 : 없음


#rag

api 안 불러와져서 기존에 저장해둔 csv 사용

api호출 대신 여기부터 실행하여 다음 코드 진행하면 됨

In [ ]:
import os
import copy
from langchain_community.document_loaders import JSONLoader,DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
#OpenAIEmbeddings는 비용나감 / llm모델이랑 별개의 작업이므로 다른 모델 사용 가능
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
df=pd.read_csv('C:/Users/tjddm/capstone/api_data.csv',encoding='utf-8')

In [ ]:
df

,title,issuedDate,category1,category2,description,tel,url,address,coordinates,charge
0,헬로독하이캣,2022-11-30,반려동물 서비스,반려동물용품,운영시간 : 월~토 10:30~21:00 | 휴무일 : 매주 일요일 | 주차가능 |...,0507-1309-9590,정보없음,(08839) 서울특별시 관악구 신림로 108,"N37.4711305, E126.937598",입장(이용료)가격 정보 : 변동 | 애견 동반 추가 요금 : 없음
1,공존하개,2022-11-30,반려동물 서비스,반려동물용품,운영시간 : 매일 00:00~24:00 | 휴무일 : 연중무휴 | 주차가능 | 반려...,0507-1348-3455,http://www.instagram.com/coexistence_dog_cat,(16836) 경기도 용인시 수지구 정평로 120,"N37.32417367, E127.093943",입장(이용료)가격 정보 : 없음 | 애견 동반 추가 요금 : 없음
2,관문체육공원,2022-11-30,반려동반여행,여행지,운영시간 : 매일 00:00~24:00 | 휴무일 : 연중무휴 | 주차가능 | 반려...,02-504-7300,정보없음,(13830) 경기도 과천시 중앙로 294,"N37.44136421, E126.996346",입장(이용료)가격 정보 : 없음 | 애견 동반 추가 요금 : 없음
3,관악그랜드약국,2022-11-30,반려의료,동물약국,"운영시간 : 월~금 09:00~20:00, 토 09:00~17:00 | 휴무일 : ...",02-877-1023,정보없음,(08846) 서울특별시 관악구 신림로 141,"N37.47159171, E126.933442",입장(이용료)가격 정보 : 변동 | 애견 동반 추가 요금 : 없음
4,공룡알화석산지,2022-11-30,반려동반여행,여행지,운영시간 : 화~일 09:00~17:00 | 휴무일 : 매주 월요일 | 주차가능 |...,031-357-3951,http://dinopia.hscity.go.kr/user/index.jsp,(18235) 경기도 화성시 송산면 공룡로 659,"N37.26222503, E126.747071",입장(이용료)가격 정보 : 없음 | 애견 동반 추가 요금 : 없음
...,...,...,...,...,...,...,...,...,...,...
8486,꽃길약국,2022-11-30,반려의료,동물약국,"운영시간 : 월~금 09:30~20:00, 토 09:30~18:00 | 휴무일 : ...",032-428-0858,정보없음,(21674) 인천광역시 남동구 소래역남로 20,"N37.39809447, E126.734742",입장(이용료)가격 정보 : 변동 | 애견 동반 추가 요금 : 없음
8487,꽃냥,2022-11-30,반려동물 서비스,미용,운영시간 : 화~일 10:00~20:00 | 휴무일 : 매주 월요일 | 주차가능 |...,0507-1356-4051,https://blog.naver.com/f-cat,(14037) 경기도 안양시 만안구 태평로60번길 42,"N37.39633325, E126.930747",입장(이용료)가격 정보 : 변동 | 애견 동반 추가 요금 : 없음
8488,꽃다비펫,2022-11-30,반려동물 서비스,미용,운영시간 : 매일 10:00~19:00 | 휴무일 : 연중무휴 | 주차가능 | 반려...,032-468-4656,http://blog.naver.com/ksbabc_,(22240) 인천광역시 미추홀구 주승로197번길 13-35,"N37.44573791, E126.691666",입장(이용료)가격 정보 : 소형견 전체미용 35000원 | 애견 동반 추가 요금 : 없음
8489,김포하나동물병원,2022-11-30,반려동물 서비스,반려동물용품,"운영시간 : 월~금 10:00~20:00, 토 10:30~19:00, 일 10:00...",031-997-8275,https://blog.naver.com/ohmoo,(10108) 경기도 김포시 관순로 4,"N37.62147372, E126.715861",입장(이용료)가격 정보 : 변동 | 애견 동반 추가 요금 : 없음


In [ ]:
missing_values = df.isna().sum()
print(missing_values)

title          0
issuedDate     0
category1      0
category2      0
description    0
tel            0
url            0
address        0
coordinates    0
charge         0
dtype: int64


##rag data 생성

사용자의 질문과 rag data의 page_content 속성과의 유사도를 통한 검색이 진행됨

즉, 검색에 필요한 정보가 모두 담긴 '하나'의 열이 필요하므로, 생성해서 page_context로 지정해준다

In [ ]:
rag_df = copy.deepcopy(df)

def generate_summary(row):
    summary = row['title'] + "은(는) " + row['address'] + "에 위치한 " + row['category2'] + " 시설입니다.\n"

    # description이 '정보없음'이 아니라면 해당 값을 사용
    if row['description'] != '정보없음':
        summary += row['description'] + ".\n"

    # charge가 '정보없음'이 아니라면 해당 값을 사용
    if row['charge'] != '정보없음':
        summary += row['charge'] + ".\n"

    # coordinates가 '정보없음'이 아니라면 해당 값을 사용
    if row['coordinates'] != '정보없음':
        summary += "좌표 : " + row['coordinates'] + ".\n"

    # url이 '정보없음'이 아니라면 해당 값을 사용
    if row['url'] != '정보없음':
        summary += "추가적인 정보는 " + row['url'] + "에서 확인하세요."

    return summary

# rag_text 추가
rag_df['rag_text'] = rag_df.apply(generate_summary, axis=1)

In [ ]:
rag_df

,title,issuedDate,category1,category2,description,tel,url,address,coordinates,charge,rag_text
0,헬로독하이캣,2022-11-30,반려동물 서비스,반려동물용품,운영시간 : 월~토 10:30~21:00 | 휴무일 : 매주 일요일 | 주차가능 |...,0507-1309-9590,정보없음,(08839) 서울특별시 관악구 신림로 108,"N37.4711305, E126.937598",입장(이용료)가격 정보 : 변동 | 애견 동반 추가 요금 : 없음,헬로독하이캣은(는) (08839) 서울특별시 관악구 신림로 108에 위치한 반려동물...
1,공존하개,2022-11-30,반려동물 서비스,반려동물용품,운영시간 : 매일 00:00~24:00 | 휴무일 : 연중무휴 | 주차가능 | 반려...,0507-1348-3455,http://www.instagram.com/coexistence_dog_cat,(16836) 경기도 용인시 수지구 정평로 120,"N37.32417367, E127.093943",입장(이용료)가격 정보 : 없음 | 애견 동반 추가 요금 : 없음,공존하개은(는) (16836) 경기도 용인시 수지구 정평로 120에 위치한 반려동물...
2,관문체육공원,2022-11-30,반려동반여행,여행지,운영시간 : 매일 00:00~24:00 | 휴무일 : 연중무휴 | 주차가능 | 반려...,02-504-7300,정보없음,(13830) 경기도 과천시 중앙로 294,"N37.44136421, E126.996346",입장(이용료)가격 정보 : 없음 | 애견 동반 추가 요금 : 없음,관문체육공원은(는) (13830) 경기도 과천시 중앙로 294에 위치한 여행지 시설...
3,관악그랜드약국,2022-11-30,반려의료,동물약국,"운영시간 : 월~금 09:00~20:00, 토 09:00~17:00 | 휴무일 : ...",02-877-1023,정보없음,(08846) 서울특별시 관악구 신림로 141,"N37.47159171, E126.933442",입장(이용료)가격 정보 : 변동 | 애견 동반 추가 요금 : 없음,관악그랜드약국은(는) (08846) 서울특별시 관악구 신림로 141에 위치한 동물약...
4,공룡알화석산지,2022-11-30,반려동반여행,여행지,운영시간 : 화~일 09:00~17:00 | 휴무일 : 매주 월요일 | 주차가능 |...,031-357-3951,http://dinopia.hscity.go.kr/user/index.jsp,(18235) 경기도 화성시 송산면 공룡로 659,"N37.26222503, E126.747071",입장(이용료)가격 정보 : 없음 | 애견 동반 추가 요금 : 없음,공룡알화석산지은(는) (18235) 경기도 화성시 송산면 공룡로 659에 위치한 여...
...,...,...,...,...,...,...,...,...,...,...,...
8486,꽃길약국,2022-11-30,반려의료,동물약국,"운영시간 : 월~금 09:30~20:00, 토 09:30~18:00 | 휴무일 : ...",032-428-0858,정보없음,(21674) 인천광역시 남동구 소래역남로 20,"N37.39809447, E126.734742",입장(이용료)가격 정보 : 변동 | 애견 동반 추가 요금 : 없음,꽃길약국은(는) (21674) 인천광역시 남동구 소래역남로 20에 위치한 동물약국 ...
8487,꽃냥,2022-11-30,반려동물 서비스,미용,운영시간 : 화~일 10:00~20:00 | 휴무일 : 매주 월요일 | 주차가능 |...,0507-1356-4051,https://blog.naver.com/f-cat,(14037) 경기도 안양시 만안구 태평로60번길 42,"N37.39633325, E126.930747",입장(이용료)가격 정보 : 변동 | 애견 동반 추가 요금 : 없음,꽃냥은(는) (14037) 경기도 안양시 만안구 태평로60번길 42에 위치한 미용 ...
8488,꽃다비펫,2022-11-30,반려동물 서비스,미용,운영시간 : 매일 10:00~19:00 | 휴무일 : 연중무휴 | 주차가능 | 반려...,032-468-4656,http://blog.naver.com/ksbabc_,(22240) 인천광역시 미추홀구 주승로197번길 13-35,"N37.44573791, E126.691666",입장(이용료)가격 정보 : 소형견 전체미용 35000원 | 애견 동반 추가 요금 : 없음,꽃다비펫은(는) (22240) 인천광역시 미추홀구 주승로197번길 13-35에 위치...
8489,김포하나동물병원,2022-11-30,반려동물 서비스,반려동물용품,"운영시간 : 월~금 10:00~20:00, 토 10:30~19:00, 일 10:00...",031-997-8275,https://blog.naver.com/ohmoo,(10108) 경기도 김포시 관순로 4,"N37.62147372, E126.715861",입장(이용료)가격 정보 : 변동 | 애견 동반 추가 요금 : 없음,김포하나동물병원은(는) (10108) 경기도 김포시 관순로 4에 위치한 반려동물용품...


In [ ]:
rag_df.at[0,'rag_text']

'헬로독하이캣은(는) (08839) 서울특별시 관악구 신림로 108에 위치한 반려동물용품 시설입니다.\n운영시간 : 월~토 10:30~21:00 | 휴무일 : 매주 일요일 | 주차가능 | 반려동물 동반가능 | 반려동물 제한사항 : 제한사항 없음.\n입장(이용료)가격 정보 : 변동 | 애견 동반 추가 요금 : 없음.\n좌표 : N37.4711305, E126.937598.\n'

In [ ]:
loader = DataFrameLoader(rag_df, page_content_column='rag_text')  #page_context를 'rag_text'열로 가진 loader 지정

docs = loader.load()
len(docs)

8491

##split

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=150) #splitter 정의
splits = text_splitter.split_documents(docs) # split

In [ ]:
print(len(splits))
print(splits[10])  #10번째 확인

8491
page_content='공복애꿀단지은(는) (11689) 경기도 의정부시 태평로 131에 위치한 미용 시설입니다.\n운영시간 : 매일 10:00~22:00 | 휴무일 : 연중무휴 | 주차가능 | 반려동물 동반가능 | 반려동물 제한사항 : 없음.\n입장(이용료)가격 정보 : 변동 | 애견 동반 추가 요금 : 없음.\n좌표 : N37.74409117, E127.051109.' metadata={'title': '공복애꿀단지', 'issuedDate': '2022-11-30', 'category1': '반려동물 서비스', 'category2': '미용', 'description': '운영시간 : 매일 10:00~22:00 | 휴무일 : 연중무휴 | 주차가능 | 반려동물 동반가능 | 반려동물 제한사항 : 없음', 'tel': '0507-1316-2125', 'url': '정보없음', 'address': '(11689) 경기도 의정부시 태평로 131', 'coordinates': 'N37.74409117, E127.051109', 'charge': '입장(이용료)가격 정보 : 변동 | 애견 동반 추가 요금 : 없음'}


In [ ]:
# 메타데이터 확인
splits[10].metadata

{'title': '공복애꿀단지',
 'issuedDate': '2022-11-30',
 'category1': '반려동물 서비스',
 'category2': '미용',
 'description': '운영시간 : 매일 10:00~22:00 | 휴무일 : 연중무휴 | 주차가능 | 반려동물 동반가능 | 반려동물 제한사항 : 없음',
 'tel': '0507-1316-2125',
 'url': '정보없음',
 'address': '(11689) 경기도 의정부시 태평로 131',
 'coordinates': 'N37.74409117, E127.051109',
 'charge': '입장(이용료)가격 정보 : 변동 | 애견 동반 추가 요금 : 없음'}

## indexing

Embedding 후, vector_db에 저장

In [ ]:
#임베딩 모델
model_name='jhgan/ko-sroberta-nli'  #임베딩 모델
model_kwargs = {"device":"cpu"}   # cpu사용
encode_kwargs={'normalize_embeddings':True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs

)

C:\Users\tjddm\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# 저장할 경로 지정
DB_PATH = "./chroma_db"

# 문서를 디스크에 저장. 저장시 persist_directory에 저장할 경로를 지정합니다.
vector_db = Chroma.from_documents(splits, embeddings, persist_directory=DB_PATH)

한번 다 돌아가면 이후에 다시 돌릴 필요 없이 불러오기만 하면 됨

In [ ]:
docs = vector_db.similarity_search("N37.74409117, E127.051109 에 위치한 시설을 알려줘")
print(len(docs))
print(docs[0].page_content)

4
진안꽃잔디동산은(는) (55425) 전라북도 진안군 진안읍 전진로 3071-31에 위치한 여행지 시설입니다.
운영시간 : 매일 09:00~18:00 | 휴무일 : 연중무휴 | 주차가능 | 반려동물 동반가능 | 반려동물 제한사항 : 목줄, 배변봉투.
입장(이용료)가격 정보 : 5000원 | 애견 동반 추가 요금 : 없음.
추가적인 정보는 http://www.mossphlox.co.kr/main#에서 확인하세요.


In [ ]:
# 현재 작업 디렉토리 확인
print(os.getcwd())

/content
